# 머신러닝 with Python 
## 09 앙상블 학습 스태킹 연습 - iris 데이터

##### 스태킹 알고리즘을 활용하여 iris를 분류하는 모형을 만드는 연습을 해본다.

(책 p.295~ 원래 실습은 유방암 양성음성 관련 데이터였으나 모형들과 관련한 의문이 남아서 다른 데이터로도 실행을 해보고자 한다.) 

---

하단부 실험에서는 이전 스태킹 실험들과 마찬가지로, 학습기 종류 자체나 학습의 순서 등을 변경하는 실험을 진행하였다.

> 배운점: 데이터마다 해당 데이터에 강력한 모형이 따로 있음을 확실히 알게 되었다.




### 데이터 불러오기

In [1]:
from sklearn import datasets
raw_iris = datasets.load_iris()

In [2]:
# 데이터 셋 내 피처 살펴보기
raw_iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

### 피처, 타깃 데이터 지정

In [3]:
X = raw_iris.data
y = raw_iris.target

### 트레이닝/테스트 데이터 분할 및 데이터 표준화

In [4]:
from sklearn.model_selection import train_test_split              # 데이터 분할을 위한 함수
from sklearn.preprocessing import StandardScaler                  # 데이터 표준화를 위한 함수

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 데이터 분할

std_scale = StandardScaler()                                      # 표준화 스케일러 선언
std_scale.fit(X_tn)                                               # 트레이닝 피처(데이터)를 기준으로 표준화 스케일러 적합

X_tn_std = std_scale.transform(X_tn)                              # 트레인, 테스트 데이터를 표준화 스케일러 처리
X_te_std = std_scale.transform(X_te)

### 데이터 학습 (베이스러너: SVM, Gnb | 메타러너: LogisticRegression)

In [5]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('svm', base_clf1),('gnb', base_clf2)],   # 개별 학습기 넣은 최종 모형 설정
                                  final_estimator = LogisticRegression(random_state = 0)) # 분류 문제를 해결해야 할 경우 StackingRegressor 사용

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

StackingClassifier(estimators=[('svm', SVC(kernel='linear', random_state=0)),
                               ('gnb', GaussianNB())],
                   final_estimator=LogisticRegression(random_state=0))

### 데이터 예측

In [6]:
pred_stacking = clf_stacking.predict(X_te_std)
print(pred_stacking)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


### 정확도 평가

In [7]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_stacking)
print(accuracy)

0.9736842105263158


### confusion matrix 확인

In [8]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_stacking)
print(conf_matrix)

[[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


### 분류 리포트 확인

In [9]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_stacking)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



### 결과

한개만 틀린 결과가 나왔다. 나쁘지 않은 성능을 보여주었는데, 과연 붓꽃 데이터에서는 나이브 베이즈는 어떤 성능을 보여줄 것인가!

# 실험- 학습기 개수 변경
## 실험1 : 베이스 학습기를 SVC만 사용했을 때

In [10]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
# from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
# base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('svm', base_clf1)],
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기- SVM만 사용했을 경우 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기- SVM만 사용했을 경우 정확도:  0.9736842105263158
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



## 실험2: 베이스 학습기를 GaussianNB만 사용했을 때

In [11]:
# from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


# base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('gnb', base_clf2)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기- 가우시안 나이브 베이즈만 사용했을 경우 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기- 가우시안 나이브 베이즈만 사용했을 경우 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00         9

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



### 위의 실험 결과

재밌는 결과가 나왔다. SVC-LR의 조합보다 GNB-LR의 조합이 더 좋은 성능을 보여주었다! 드디어 가우시안 나이브 베이즈도 잘하는 게 있다고 증명을 한 것이다! (?)

물론 각각 만 개도 안되는 데이터들로 결론을 내리기에는 조악하긴 하지만, 앞전과는 다른, 의외의 결과인 것이다.

---

# 실험3: 학습기 순서 변경

In [12]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('gnb', base_clf2), ('svc', base_clf1)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("베이스 학습기 순서 변경 후 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
# print("========================================================")

베이스 학습기 순서 변경 후 정확도:  0.9736842105263158
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



### 결과

순서가 상관이 없다는 것이 증명이 되었음에도 그저 돌려보았는데 뭐 그런가보다 싶을 때까지 코드가 익숙해졌으면 좋겠어서.

# 실험- 기본 학습기와 메타 학습기 조합 변경

베이스 학습기들의 결과가 나온 것을 메타 학습기가 받는다고 했을 때, 

같은 종류의 학습기가 순차적으로 데이터를 받아 분류한다면 결과가 어떨지에 대한 궁금증과,

세 가지 분류 알고리즘들의 조합에 따라 이 데이터 셋에서는 어떤 성능차이가 있을지 궁금해졌다.

## 실험4: 셋 중 메타 학습기를 변경했을 때

In [13]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형

clf_svc = svm.SVC(kernel = 'linear', random_state = 0)
clf_gnb = GaussianNB()
clf_lr = LogisticRegression(random_state = 0)


# 메타 학습기가 SVC 일 때
clf_stacking = StackingClassifier(estimators = [('gnb', clf_gnb), ('lr', clf_lr)], 
                                  final_estimator = svm.SVC(kernel = 'linear', random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기: ", "SVM")
print("베이스 학습기: ", "Gaussian Naive Bayes, Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# 메타 학습기가 Naive Bayes 일 때
clf_stacking = StackingClassifier(estimators = [('svc', clf_svc), ('lr', clf_lr)], final_estimator = GaussianNB()) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기: ", "Gaussian Naive Bayes")
print("베이스 학습기: ", "SVM, Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)

메타 학습기:  SVM
베이스 학습기:  Gaussian Naive Bayes, Logistic Regression
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00         9

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38

메타 학습기:  Gaussian Naive Bayes
베이스 학습기:  SVM, Logistic Regression
모델 정확도:  0.9736842105263158
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



### 결과

이번에는 베이스 학습기가 가우시안 나이브 베이즈가 포함된 상태가 성능이 더 좋게 나왔다. 그렇다면 학습기 모형이 같은 모형일때 성능은 어떨까?

## 실험5: 베이스 학습기와 메타 학습기가 같은 모형일 때

In [14]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형

clf_svc = svm.SVC(kernel = 'linear', random_state = 0) 
clf_gnb = GaussianNB()
clf_lr = LogisticRegression(random_state = 0)


# LogisticRegression
clf_stacking = StackingClassifier(estimators = [('lr', clf_lr)], 
                                  final_estimator = LogisticRegression(random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "Logistic Regression")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# SVM
clf_stacking = StackingClassifier(estimators = [('svc', clf_svc)],
                                  final_estimator = svm.SVC(kernel = 'linear', random_state = 0)) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "SVM")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)
print("========================================================")


# Gaussian Naive Bayes
clf_stacking = StackingClassifier(estimators = [('gnb', clf_gnb)], 
                                  final_estimator = GaussianNB()) 

clf_stacking.fit(X_tn_std, y_tn)

pred_stacking = clf_stacking.predict(X_te_std)
accuracy = accuracy_score(y_te, pred_stacking)

conf_matrix = confusion_matrix(y_te, pred_stacking)
class_report = classification_report(y_te, pred_stacking)

print("메타 학습기 = 베이스 학습기: ", "Gaussian Naive Bayes")
print("모델 정확도: ", accuracy)
print("모델 분류 리포트")
print(class_report)

메타 학습기 = 베이스 학습기:  Logistic Regression
모델 정확도:  0.9736842105263158
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

메타 학습기 = 베이스 학습기:  SVM
모델 정확도:  0.9736842105263158
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

메타 학습기 = 베이스 학습기:  Gaussian Naive Bayes
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1

### 결과

붓꽃 데이터에서는 Gaussian Naive Bayes가 LogisticRegression과 SupportVectorMachine보다 더 나은 성능을 보여주었다. 

적은 데이터이고 다른 여러 옵션들(조건을 같게 할 수 있는 상세한 튜닝)을 만질 수 있는 수준이 되지 못함에 아쉽지만 이전의 와인이나 유방암 데이터와는 다른 결과간 나온 것이 재미있다.

다른 데이터로도 스태킹을 실험해보고 싶다.